In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/american-express-1/XBG_918_features_wo_imputation_fold4.xgb
/kaggle/input/american-express-1/XBG_918_features_wo_imputation_fold3.xgb
/kaggle/input/american-express-1/__results__.html
/kaggle/input/american-express-1/XBG_918_features_wo_imputation_fold0.xgb
/kaggle/input/american-express-1/__notebook_source__.ipynb
/kaggle/input/american-express-1/XBG_918_features_wo_imputation_fold1.xgb
/kaggle/input/american-express-1/__notebook__.ipynb
/kaggle/input/american-express-1/__output__.json
/kaggle/input/american-express-1/XBG_918_features_wo_imputation_fold2.xgb
/kaggle/input/american-express-1/custom.css
/kaggle/input/amex-credit/918_features_wo_imputation_2.parquet
/kaggle/input/amex-credit/test_918_features_wo_imputation_2.csv
/kaggle/input/amex-default-prediction/sample_submission.csv
/kaggle/input/amex-default-prediction/train_data.csv
/kaggle/input/amex-default-prediction/test_data.csv
/kaggle/input/amex-default-prediction/train_labels.csv


In [2]:
def read_file(path, cols = None):
    print(cols)
    if cols is not None: 
        df = pd.read_csv(path, usecols = cols, compression = 'gzip')
    else:
        df = pd.read_csv(path, compression = 'gzip')
        
    df.S_2 = df.to_datetime(df.S_2)
    return df

def read_file1(path, cols = None):
    df = pd.read_csv(path, usecols = ['customer_ID'], compression = 'gzip')
    
#     df.S_2 = df.to_datetime(df.S_2)
    return df

In [3]:
def get_rows(customers, test, NUM_PARTS = 4, verbose = ''):
    chunk = len(customers) // NUM_PARTS
    if verbose != '':
        print(f'We will process {verbose} data as {NUM_PARTS} separate parts.')
        print(f'There will be {chunk} customers in each part (except the last part).')
        print('Below are number of rows in each part:')
    rows = []
    
    for k in range(NUM_PARTS):
        if k==NUM_PARTS-1: cc = customers[k*chunk:]
        else: cc = customers[k*chunk:(k+1)*chunk]
        s = test.loc[test.customer_ID.isin(cc)].shape[0]
        rows.append(s)        
    return rows,chunk

NUM_PARTS = 10
TEST_PATH = '/kaggle/input/amex-credit/test_918_features_wo_imputation_2.csv'

test = read_file1(path = TEST_PATH, cols = ['customer_ID','S_2'])
customers = test[['customer_ID']].drop_duplicates().sort_index().values.flatten()
rows,num_cust = get_rows(customers, test[['customer_ID']], NUM_PARTS = NUM_PARTS, verbose = 'test')


We will process test data as 10 separate parts.
There will be 92462 customers in each part (except the last part).
Below are number of rows in each part:


In [ ]:
import time
import gc
import xgboost as xgb

In [ ]:
def get_num_rows(df):
    print("Started reading file")
    start = time.time()
    data = pd.read_csv(df, usecols = ["customer_ID"], compression = 'gzip')
    print(f"Time taken to read file {time.time() - start}")
    customers = data[['customer_ID']].drop_duplicates().sort_index().values.flatten()
    size = len(data)
    del data
    return customers, size

TEST_PATH = '/kaggle/input/amex-credit/test_918_features_wo_imputation_2.csv'
customers, total_samples = get_num_rows(TEST_PATH)

In [ ]:
chunk_size = 1e4
rounds = total_samples // chunk_size + 1
FOLDS = 5

In [ ]:
test_preds = []

data = pd.read_csv(TEST_PATH, chunksize = chunk_size, compression = 'gzip')

for r, chunk in enumerate(data): 
    print(f"Reading chunk: {r}")
    start = time.time()
    features = chunk.columns[1:]
    X_test = chunk[features]
    print(f"Shape of test data: {X_test.shape}")
    dtest = xgb.DMatrix(data = X_test)
    
    del chunk, X_test
    gc.collect()
    
    preds = []
    model = xgb.Booster()
    model.load_model(f'/kaggle/input/american-express-1/XBG_918_features_wo_imputation_fold0.xgb')
    preds = model.predict(dtest)
    for fold in range(1, FOLDS):
        model.load_model(f'/kaggle/input/american-express-1/XBG_918_features_wo_imputation_fold{fold}.xgb')
        preds += model.predict(dtest)
        
    preds /= FOLDS
    test_preds.append(preds)
    
    del dtest, model
    gc.collect()
    print(f"Total processing time: {time.time() - start}")
    

In [ ]:
test_preds = np.concatenate(test_preds)

In [ ]:
np.save('918_features_wo_imputation.npy', test_preds)


In [ ]:
test = pd.DataFrame(index = customers, data = {'prediction': test_preds})

In [ ]:
sub = pd.read_csv("/kaggle/input/amex-default-prediction/sample_submission.csv")[['customer_ID']]
sub = sub.set_index('customer_ID')

In [ ]:
sub = sub.merge(test[['prediction']], left_index = True, right_index = True, how = 'left')
# sub = sub.reset_index(drop=True)


In [ ]:
sub = sub.reset_index()

In [ ]:
sub.to_csv('submission.csv', index = False)
print(f'Submission file shape: {sub.shape}')
sub.head()